# Push Prices Handler

This module receives price changes from Module 2 or Module 3 and pushes them to the MaxAB API.

## Workflow
1. Receive price data from Module 2 (daily reset) or Module 3 (periodic adjustments)
2. Filter to only SKUs where price actually changed
3. Aggregate prices by cohort using stock-weighted average
4. Expand to all packing units (multiply by basic_unit_count)
5. Apply visibility rules (remove minimum PU for multi-PU products)
6. Split into chunks and upload via MaxAB API

## Required Input Columns
- `product_id`: Product ID
- `sku`: SKU name
- `new_price`: New price to set (basic unit price)
- `warehouse_id`: Warehouse ID
- `cohort_id`: Cohort ID for pricing
- `stocks`: Current stock level (used for weighted price aggregation)
- `current_price`: Current price (for filtering changed prices)

## Output
- Uploads pricing sheets to MaxAB API per cohort
- Returns summary dict with counts of pushed/failed/skipped


In [ ]:
# =============================================================================
# IMPORTS
# =============================================================================
import pandas as pd
import numpy as np
import json
import time
import base64
import os
from datetime import datetime
import pytz

# AWS for secrets management
import boto3
from botocore.exceptions import ClientError

# HTTP requests for API calls
import requests

# Progress bar for chunk uploads
from tqdm import tqdm

# Google Sheets integration
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Import setup_environment_2 for Google Sheets credentials
import sys
sys.path.append('..')
import setup_environment_2

# Cairo timezone
CAIRO_TZ = pytz.timezone('Africa/Cairo')
CAIRO_NOW = datetime.now(CAIRO_TZ)

# =============================================================================
# CONFIGURATION
# =============================================================================
CHUNK_SIZE_DEFAULT = 4000   # Default chunk size for API uploads
CHUNK_SIZE_SPECIAL = 2000   # Smaller chunk size for specific cohorts (e.g., cohort 61)
UPLOAD_DIR = 'uploads'      # Directory to save full upload files
MANUAL_DIR = 'manual'       # Directory to save chunk files

# Google Sheets configuration
GSHEET_NAME = 'Demand Based Dynamic Pricing'
DISABLE_PU_SHEET = 'disable_pu_visibility'

print(f"Push Prices Handler loaded at {CAIRO_NOW.strftime('%Y-%m-%d %H:%M:%S')} Cairo time")


In [ ]:
# =============================================================================
# AWS & API FUNCTIONS
# =============================================================================

def get_secret(secret_name: str) -> str:
    """
    Retrieve a secret from AWS Secrets Manager.
    
    This function connects to AWS Secrets Manager to fetch sensitive credentials
    (like API keys, passwords) securely without hardcoding them in the code.
    
    Args:
        secret_name: The name/path of the secret in AWS Secrets Manager
                    (e.g., "prod/pricing/api/")
    
    Returns:
        Secret string (JSON format) or decoded binary
        
    Raises:
        ClientError: If the secret cannot be retrieved (permissions, not found, etc.)
    """
    region_name = "us-east-1"
    session = boto3.session.Session()
    client = session.client(service_name='secretsmanager', region_name=region_name)

    try:
        response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        error_code = e.response['Error']['Code']
        error_messages = {
            'DecryptionFailureException': "Can't decrypt secret using provided KMS key",
            'InternalServiceErrorException': "Server-side error occurred",
            'InvalidParameterException': "Invalid parameter value provided",
            'InvalidRequestException': "Invalid request for current resource state",
            'ResourceNotFoundException': "Requested resource not found"
        }
        if error_code in error_messages:
            print(f"AWS Error: {error_messages[error_code]}")
        raise e
    
    if 'SecretString' in response:
        return response['SecretString']
    return base64.b64decode(response['SecretBinary'])


def get_access_token(url: str, client_id: str, client_secret: str) -> str:
    """
    Get OAuth2 access token for MaxAB API authentication.
    
    Uses password grant type to exchange credentials for an access token.
    The token is used in subsequent API calls for authorization.
    
    Args:
        url: The OAuth token endpoint URL
        client_id: OAuth client ID for the application
        client_secret: OAuth client secret for the application
    
    Returns:
        Access token string to be used in Authorization header
    """
    response = requests.post(
        url,
        data={
            "grant_type": "password",
            "username": API_USERNAME,
            "password": API_PASSWORD
        },
        auth=(client_id, client_secret),
    )
    return response.json()["access_token"]


def _get_api_token() -> str:
    """
    Get a fresh API token for MaxAB API requests.
    
    This is a convenience wrapper that calls get_access_token with
    the correct MaxAB SSO endpoint and client credentials.
    
    Returns:
        Fresh access token string
    """
    return get_access_token(
        'https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token',
        'main-system-externals',
        API_SECRET
    )


def post_prices(cohort_id: int, file_name: str) -> requests.Response:
    """
    Upload a pricing Excel sheet to MaxAB API for a specific cohort.
    
    This function:
    1. Gets a fresh API token
    2. Prepares the file for multipart upload
    3. POSTs to the cohort pricing endpoint
    
    Args:
        cohort_id: The target cohort ID to upload prices for
        file_name: Path to the Excel file containing pricing data
                  Expected columns: Product ID, Product Name, Packing Unit ID, 
                                   Price, Visibility (YES/NO)
    
    Returns:
        requests.Response object with API response
        Check response.content for '{"success":true}' to verify upload
    """
    token = _get_api_token()
    url = f"https://api.maxab.info/main-system/api/admin-portal/cohorts/{cohort_id}/pricing"
    
    files = [('sheet', (file_name, open(file_name, 'rb'), 
              'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'))]
    headers = {'Authorization': f'bearer {token}'}
    
    return requests.post(url, headers=headers, data={}, files=files)


In [ ]:
# =============================================================================
# API CREDENTIALS INITIALIZATION
# =============================================================================
# Load API credentials from AWS Secrets Manager
# These are used by get_access_token() to authenticate with MaxAB API

pricing_api_secret = json.loads(get_secret("prod/pricing/api/"))
API_USERNAME = pricing_api_secret["egypt_username"]
API_PASSWORD = pricing_api_secret["egypt_password"]
API_SECRET = pricing_api_secret["egypt_secret"]

print("✓ API credentials loaded successfully")

# =============================================================================
# GOOGLE SHEETS CLIENT INITIALIZATION
# =============================================================================
# Initialize Google Sheets client for reading disable_pu_visibility data

GSHEET_SCOPE = [
    "https://spreadsheets.google.com/feeds",
    'https://www.googleapis.com/auth/spreadsheets',
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

gsheet_creds = ServiceAccountCredentials.from_json_keyfile_dict(
    json.loads(setup_environment_2.get_secret("prod/maxab-sheets")), 
    GSHEET_SCOPE
)
gsheet_client = gspread.authorize(gsheet_creds)

print("✓ Google Sheets client initialized")


In [ ]:
# =============================================================================
# GOOGLE SHEETS DATA LOADING
# =============================================================================

def get_disable_pu_visibility() -> pd.DataFrame:
    """
    Load disable PU visibility data from Google Sheets.
    
    This function reads from the 'disable_pu_visibility' sheet in the 
    'Demand Based Dynamic Pricing' Google Sheet. This sheet contains products 
    whose minimum packing unit should be hidden from retailers.
    
    Sheet columns:
        - cohort_id: Cohort identifier (dropped - not needed)
        - product_id: Product identifier
        - packing_unit_id: Packing unit identifier (kept for reference)
    
    Returns:
        DataFrame with columns: product_id, packing_unit_id
        Duplicates are removed based on all columns.
    """
    print("Loading disable_pu_visibility from Google Sheets...")
    
    # Open the Google Sheet and get the specific worksheet
    sheet = gsheet_client.open(GSHEET_NAME).worksheet(DISABLE_PU_SHEET)
    
    # Convert to DataFrame
    df = pd.DataFrame(sheet.get_all_records())
    
    if df.empty:
        print("  ⚠️ No data in disable_pu_visibility sheet")
        return pd.DataFrame(columns=['product_id', 'packing_unit_id'])
    
    # Drop cohort_id column (not needed for visibility logic)
    if 'cohort_id' in df.columns:
        df = df.drop(columns=['cohort_id'])
    
    # Drop duplicates
    df = df.drop_duplicates()
    
    # Add remove_min flag (all rows in this sheet should have visibility disabled)
    df['remove_min'] = 1
    
    print(f"  ✓ Loaded {len(df)} products to disable min PU visibility")
    
    return df


# =============================================================================
# PRICE HELPER FUNCTIONS
# =============================================================================

def round_price(price):
    """
    Round price to the nearest 0.25 EGP.
    
    MaxAB pricing system uses quarter-pound increments (0.25 EGP).
    This function ensures all prices conform to this requirement.
    
    Formula: round(price * 4) / 4
    - Multiply by 4 to convert to quarter units
    - Round to nearest integer
    - Divide by 4 to convert back to EGP
    
    Examples:
        10.13 -> 10.25
        10.37 -> 10.50
        10.62 -> 10.75
        10.87 -> 11.00
    
    Args:
        price: The price value to round
        
    Returns:
        Price rounded to nearest 0.25 EGP
    """
    return round(price * 4) / 4

In [ ]:
# =============================================================================
# DATA PREPARATION FUNCTIONS
# =============================================================================

def prices_preparation(df_prices: pd.DataFrame, pus: pd.DataFrame) -> pd.DataFrame:
    """
    Prepare price data for API upload by aggregating and expanding to packing units.
    
    This function performs a critical transformation:
    1. Since each cohort may have multiple warehouses, we need to calculate a 
       single price per product-cohort combination
    2. We use stock-weighted average: warehouses with more stock have more 
       influence on the final price
    3. Then we expand to all packing units (multiply by basic_unit_count)
    
    Stock-Weighted Price Calculation:
    ---------------------------------
    For each product in a cohort:
    - If warehouse A has 70% of stock and price 10 EGP
    - And warehouse B has 30% of stock and price 12 EGP
    - Final price = (0.7 * 10) + (0.3 * 12) = 10.6 EGP
    
    If stocks are 0 or null, we fall back to equal weights (1/num_warehouses).
    
    Args:
        df_prices: DataFrame with columns:
            - product_id: Product identifier
            - sku: SKU name  
            - cohort_id: Target cohort
            - warehouse_id: Source warehouse
            - stocks: Stock quantity (for weighting)
            - new_price: New basic unit price
            
        pus: Packing units DataFrame with columns:
            - product_id: Product identifier
            - packing_unit_id: Packing unit ID
            - basic_unit_count: Number of basic units in this packing unit
    
    Returns:
        DataFrame with columns:
            - product_id, sku, cohort_id: Identifiers
            - final_price: Stock-weighted average price (basic unit)
            - packing_unit_id: Packing unit ID
            - basic_unit_count: Units per pack
            - final_pu_price: final_price * basic_unit_count
    """
    # Step 1: Calculate total stocks per product-cohort for weighting
    df_prices['cohort_total_stocks'] = df_prices.groupby(['product_id', 'cohort_id'])['stocks'].transform('sum')
    
    # Step 2: Count unique warehouses per product-cohort (fallback for zero stocks)
    df_prices['cohort_total_whs'] = df_prices.groupby(['product_id', 'cohort_id'])['warehouse_id'].transform('nunique')
    
    # Step 3: Calculate stock contribution (weight) for each warehouse
    # If total stocks = 0, use equal weights (1 / number of warehouses)
    df_prices['stocks_cntrb'] = df_prices['stocks'] / df_prices['cohort_total_stocks']
    df_prices['stocks_cntrb'] = df_prices['stocks_cntrb'].fillna(1 / df_prices['cohort_total_whs'])
    
    # Step 4: Calculate weighted price contribution from each warehouse
    df_prices['final_price'] = df_prices['stocks_cntrb'] * df_prices['new_price']
    
    # Step 5: Aggregate to get single price per product-cohort
    df_final = df_prices.groupby(['product_id', 'sku', 'cohort_id'])['final_price'].sum().reset_index()
    
    # Step 6: Round to nearest 0.25 EGP
    df_final['final_price'] = df_final['final_price'].apply(round_price)
    
    # Step 7: Expand to all packing units
    df_final = df_final.merge(pus, on=['product_id'])
    
    # Step 8: Calculate packing unit price (basic price * units per pack)
    df_final['final_pu_price'] = df_final['final_price'] * df_final['basic_unit_count']
    
    return df_final

NameError: name 'pd' is not defined

In [ ]:
# =============================================================================
# MAIN PUSH PRICES FUNCTION
# =============================================================================

# Mode constants for clarity
MODE_TESTING = 'testing'  # Prepare files but DON'T upload to API
MODE_LIVE = 'live'        # Prepare files AND upload to API

def push_prices(df_prices: pd.DataFrame, pus: pd.DataFrame, 
                remove_min_pu: pd.DataFrame = None,
                source_module: str = 'unknown',
                mode: str = 'testing',
                skip_cohorts: list = None) -> dict:
    """
    Main entry point: Push price changes to MaxAB API.
    
    This function orchestrates the entire price upload workflow:
    1. Filter to only changed prices
    2. Prepare data (stock-weighted aggregation, packing unit expansion)
    3. Apply visibility rules (hide min PU for multi-PU products)
    4. Split into chunks and upload via API (only if mode='live')
    
    ⚠️ IMPORTANT: Default mode is 'testing' - prices will NOT be pushed!
    Set mode='live' to actually upload prices to the API.
    
    Visibility Rules:
    -----------------
    For products with multiple packing units (e.g., single bottle + pack of 6):
    - The minimum PU (ind=1, smallest basic_unit_count) is hidden (Visibility=NO)
    - This prevents retailers from buying single units when packs are available
    - Products in remove_min_pu list also get their min PU hidden
    
    Parameters:
    -----------
    df_prices : pd.DataFrame
        DataFrame with price recommendations from Module 2 or 3
        Required columns: product_id, sku, new_price, warehouse_id, cohort_id, 
                         stocks, current_price
        
    pus : pd.DataFrame
        Packing units lookup table
        Required columns: product_id, packing_unit_id, basic_unit_count
        
    remove_min_pu : pd.DataFrame, optional
        Products whose min PU should always be hidden
        Required columns: product_id
        
    source_module : str
        Identifier for calling module ('module_2' or 'module_3')
        Used in logging and file naming
        
    mode : str
        'testing' (default) - Prepare and save files, but DON'T upload to API
        'live' - Prepare files AND upload to MaxAB API
        
    skip_cohorts : list, optional
        List of cohort IDs to skip (e.g., cohorts where cart rules failed)
        Prices for these cohorts will not be pushed
        
    Returns:
    --------
    dict with results:
        - total_received: Input row count
        - price_changes: Rows with actual price changes
        - skipped: Rows with no change
        - pushed: Successfully uploaded (or would be in testing mode)
        - failed: Upload failures
        - source_module: Which module called this
        - timestamp: When upload completed
        - mode: 'testing' or 'live'
        - skipped_cohorts: Cohorts that were skipped due to cart rule failures
    """
    
    # Validate mode parameter
    if mode not in [MODE_TESTING, MODE_LIVE]:
        print(f"⚠️ Invalid mode '{mode}'. Using 'testing' mode.")
        mode = MODE_TESTING
    
    # Print clear mode indicator
    print(f"\n{'🧪' if mode == MODE_TESTING else '🚀'} MODE: {mode.upper()}")
    if mode == MODE_TESTING:
        print("   Files will be prepared but NOT uploaded to API")
    else:
        print("   Files will be prepared AND uploaded to API")
    
    # Initialize skip_cohorts if not provided
    if skip_cohorts is None:
        skip_cohorts = []
    
    if skip_cohorts:
        print(f"   ⚠️ Skipping cohorts (cart rules failed): {skip_cohorts}")
    
    # Load disable PU visibility data from Google Sheets if not provided
    if remove_min_pu is None:
        remove_min_pu = get_disable_pu_visibility()
    
    # Initialize result tracking
    result = {
        'total_received': len(df_prices),
        'price_changes': 0,
        'pushed': 0,
        'failed': 0,
        'skipped': 0,
        'source_module': source_module,
        'timestamp': datetime.now(CAIRO_TZ).strftime('%Y-%m-%d %H:%M:%S'),
        'mode': mode,
        'skipped_cohorts': skip_cohorts.copy()  # Cohorts skipped due to cart rule failures
    }
    
    # =========================================================================
    # STEP 1: Validate input
    # =========================================================================
    if df_prices.empty:
        print(f"⚠️ No data received from {source_module}")
        return result
    
    # Filter to only rows where price actually changed
    df_to_push = df_prices[
        (df_prices['new_price'].notna()) & 
        (df_prices['new_price'] != df_prices['current_price'])
    ].copy()
    
    result['price_changes'] = len(df_to_push)
    result['skipped'] = len(df_prices) - len(df_to_push)
    
    if df_to_push.empty:
        print(f"ℹ️ No price changes to push from {source_module}")
        return result
    
    print(f"\n{'='*60}")
    print(f"PUSH PRICES - Source: {source_module}")
    print(f"{'='*60}")
    print(f"Total received: {result['total_received']}")
    print(f"Price changes to push: {result['price_changes']}")
    print(f"Skipped (no change): {result['skipped']}")
    
    # Check required columns
    required_cols = ['product_id', 'sku', 'new_price', 'warehouse_id', 'cohort_id', 'stocks']
    missing_cols = [c for c in required_cols if c not in df_to_push.columns]
    if missing_cols:
        print(f"❌ ERROR: Missing required columns: {missing_cols}")
        result['failed'] = len(df_to_push)
        return result
    
    # =========================================================================
    # STEP 2: Log price change summary
    # =========================================================================
    print(f"\nPrice changes summary:")
    if 'current_price' in df_to_push.columns:
        increases = len(df_to_push[df_to_push['new_price'] > df_to_push['current_price']])
        decreases = len(df_to_push[df_to_push['new_price'] < df_to_push['current_price']])
        print(f"  Increases: {increases}")
        print(f"  Decreases: {decreases}")
    
    # =========================================================================
    # STEP 3: Prepare data (aggregate by cohort, expand to packing units)
    # =========================================================================
    push_data = df_to_push[required_cols].copy()
    final_data = prices_preparation(push_data, pus)
    
    # Store for debugging/inspection
    global prices_to_push
    prices_to_push = final_data
    
    print(f"\n📋 Prepared {len(final_data)} packing unit prices")
    
    # =========================================================================
    # STEP 4: Apply visibility rules (hide min PU for multi-PU products)
    # =========================================================================
    # Create index for packing unit ordering (1 = smallest/min PU)
    final_data.sort_values(['cohort_id', 'product_id', 'basic_unit_count']).reset_index(drop=True)
    final_data['ind'] = 1
    final_data['ind'] = final_data.groupby(['cohort_id', 'product_id'])['ind'].cumsum()
    
    # Apply remove_min rules if provided
    if remove_min_pu is not None and not remove_min_pu.empty:
        if 'remove_min' not in remove_min_pu.columns:
            remove_min_pu['remove_min'] = 1
        final_data = final_data.merge(
            remove_min_pu[['product_id', 'remove_min']], 
            on='product_id', 
            how='left'
        )
    else:
        final_data['remove_min'] = np.nan
    
    
    # =========================================================================
    # STEP 5: Upload to API per cohort
    # =========================================================================
    # Ensure output directories exist
    os.makedirs(UPLOAD_DIR, exist_ok=True)
    os.makedirs(MANUAL_DIR, exist_ok=True)
    
    total_pushed = 0
    total_failed = 0
    
    for cohort in final_data.cohort_id.unique():
        print(f"\n{'='*50}")
        print(f"Processing cohort: {cohort}")
        print('='*50)
        
        # Skip cohorts where cart rules failed
        if cohort in skip_cohorts:
            print(f"  ⏭️ SKIPPED - Cart rules failed for this cohort")
            continue
        
        # Filter to this cohort
        cohort_data = final_data[final_data['cohort_id'] == cohort].copy()
        
        # Prepare output DataFrame with API-expected columns
        out = cohort_data[['product_id', 'sku', 'packing_unit_id', 'final_pu_price', 'ind', 'remove_min']].copy()
        out.columns = ['Product ID', 'Product Name', 'Packing Unit ID', 'Price', 'ind', 'remove_min']
        
        # Set visibility based on remove_min flag
        out['Visibility (YES/NO)'] = 'YES'
        out.loc[(out['ind'] == 1) & (out['remove_min'] == 1), 'Visibility (YES/NO)'] = 'NO'
        
        # Drop helper columns and duplicates
        out = out.drop(columns=['ind', 'remove_min']).drop_duplicates()
        
        # Add required empty columns for API
        out['Execute At (format:dd/mm/yyyy HH:mm)'] = None
        out['Tags'] = None
        
        # Filter out invalid prices
        out = out[out['Price'] > 1].reset_index(drop=True)
        
        if len(out) == 0:
            print(f"  No valid prices for cohort {cohort}")
            continue
        
        # Save full file for reference
        file_name_ = f'{UPLOAD_DIR}/{source_module}_{cohort}.xlsx'
        out.to_excel(file_name_, index=False)
        print(f"  Saved: {file_name_} ({len(out)} rows)")
        time.sleep(2)
        
        # In testing mode, skip the actual API upload
        if mode == MODE_TESTING:
            print(f"  🧪 [TESTING] Would upload {len(out)} prices (skipped)")
            total_pushed += len(out)
            continue
        
        # Split into chunks for API upload
        chunk_size = CHUNK_SIZE_SPECIAL if cohort == 61 else CHUNK_SIZE_DEFAULT
        chunks = [out[i:i + chunk_size] for i in range(0, len(out), chunk_size)]
        print(f"  Split into {len(chunks)} chunks (size: {chunk_size})")
        
        # Save and upload chunks
        fileslist = []
        for i, chunk in tqdm(enumerate(chunks), total=len(chunks), desc="  Saving chunks"):
            output_file = f'{MANUAL_DIR}/output_{cohort}_chunk_{i + 1}.xlsx'
            fileslist.append(output_file)
            chunk.to_excel(output_file, index=False)
        
        # Upload each chunk
        print("  Uploading...")
        upload_success = True
        
        for file in fileslist:
            chunk_num = file.split('chunk_')[1].split('.xls')[0]
            response = post_prices(cohort, file)
            
            if '"success":true' in str(response.content).lower():
                print(f"    ✓ Chunk {chunk_num} uploaded successfully")
                total_pushed += len(pd.read_excel(file))
            else:
                print(f"    ✗ ERROR chunk {chunk_num}")
                print(f"      Response: {response.content}")
                upload_success = False
                total_failed += len(pd.read_excel(file))
                break
        
        if not upload_success:
            print(f"  Upload failed for cohort {cohort}")
    
    # =========================================================================
    # STEP 6: Final summary
    # =========================================================================
    result['pushed'] = total_pushed
    result['failed'] = total_failed
    
    print(f"\n{'='*60}")
    if mode == MODE_TESTING:
        print("🧪 TESTING MODE COMPLETE - NO PRICES WERE UPLOADED")
    else:
        print("🚀 UPLOAD COMPLETE")
    print(f"{'='*60}")
    print(f"Mode: {mode}")
    print(f"Total prepared: {total_pushed}")
    if mode == MODE_LIVE:
        print(f"Total failed: {total_failed}")
    
    return result


# Global variable to store prepared prices for debugging/inspection
prices_to_push = pd.DataFrame()
